In [1]:
from pathlib import Path
import pandas as pd
import json
import numpy as np
# import shap

In [2]:
PATH = Path(r'C:\Users\niti.mishra\Documents\2_TDMDAL\projects\covid_tests\covid_tests')
# PATH = Path(r'/Users/peaceforlives/Documents/Projects/covid_symptoms')
DATA = Path.joinpath(PATH, 'data', 'old')
RESULTS = Path.joinpath(PATH, 'results')

## DATA ##

In [17]:
covid_data = pd.read_csv(Path.joinpath(DATA, 'RawData_IMOH_Sep_20.csv')) 

In [18]:
y = covid_data['testresult']
cols = ['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache', 'sixtiesplus', 'Gender', 'contact', 'abroad', 'Validation']
covid_data = covid_data[cols]

In [19]:
covid_data.index = covid_data['Validation']
covid_data.drop('Validation', inplace=True, axis=1)
covid_data.head()

,cough,fever,sore_throat,shortness_of_breath,head_ache,sixtiesplus,Gender,contact,abroad
Validation,,,,,,,,,
Training,0,1,0,0,0,1,1,0,0
Validation,0,1,0,0,0,0,0,0,0
Test,1,0,0,0,1,0,1,0,0
Test,1,0,0,0,0,0,0,1,0
Training,0,0,0,0,1,0,1,0,0


In [11]:
test = list(covid_data.sum(axis=1))

In [17]:
from collections import Counter

Counter(y)

Counter({1: 47238, 0: 35681})

In [21]:
(y.value_counts()/len(y))*100

1    56.968849
0    43.031151
Name: testresult, dtype: float64

In [22]:
test = {'cough': '0', 'fever': '1'}

# [[key for key in test[0].keys()], *[list(idx.values()) for idx in test]]

In [23]:
print("{" + "\n".join("{!r}: {!r},".format(k, v) for k, v in test.items()) + "}")

{'cough': '0',
'fever': '1',}


In [24]:
train = covid_data.index.get_loc('Training')
valid = covid_data.index.get_loc('Validation')
test = covid_data.index.get_loc('Test')

In [25]:
X_train = covid_data[train].values.tolist()
y_train = y[train].values.tolist()
print(len(y_train))

X_valid = covid_data[valid].values.tolist()
y_valid = y[valid].values.tolist()
print(len(y_valid))

X_test = covid_data[test].values.tolist()
y_test = y[test].values.tolist()
print(len(y_test))

41464
20744
20711


In [22]:
import numpy as np
import time
from sklearn.pipeline import Pipeline
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, auc, roc_auc_score



In [20]:
parameters = [
    {'n_estimators':range(20,81,10),
        'max_depth':range(5,16,2), 
        'max_features':range(7,20,2),
        'min_samples_split':range(200,1001,200),
        'learning_rate':[.001,0.01,.1],
        'subsample':[0.6,0.7,0.75,0.8,0.85,0.9],
        }
]

In [32]:
rf = RandomForestClassifier(max_depth=10, min_samples_leaf=4,
                                        min_samples_split=10,
                                        random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
roc_auc_score(y_test, y_pred)

0.7787989969376632

In [ ]:
y_pred = rf.predict(X_test)
roc_auc_score(y_test, y_pred)

In [119]:
lr = LogisticRegression(random_state=42, penalty='none')
lr.fit(X_train, y_train)

LogisticRegression(penalty='none', random_state=42)

In [121]:
y_pred = lr.predict(X_test)
roc_auc_score(y_test, y_pred)

0.7743491881068725

In [122]:
lr.coef_

array([[-0.78111456,  0.46842223,  0.87043669,  1.03072509,  1.27444187,
         0.3634671 , -0.10938874,  0.61187191, -2.49559097]])

In [20]:
# # rename columns
# cols = ['test_date']
# cols.extend(covid_data.columns[1:] )
# covid_data.columns = cols

# # select columns
# cols = ['test_date', 'cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache',
#         'testresult', 'sixtiesplus', 'Gender', 'contact', 'abroad', 'Validation']
# covid_data = covid_data[cols]

# covid_data['test_date'] = pd.to_datetime(covid_data['test_date'], format='%Y-%m-%d')

# covid_data.head()

## RESULTS ##

In [8]:
RESULTS = Path.joinpath(PATH, 'results') #, 'no_missingdata',)
str(RESULTS)+'\\results.json'

'C:\\Users\\niti.mishra\\Documents\\2_TDMDAL\\projects\\covid_tests\\covid_tests\\results\\results.json'

In [9]:
res = pd.read_json(str(RESULTS)+'\\results.json', orient='records', lines=True)

In [10]:
res.set_index('time', inplace=True)

In [11]:
res

,name,model,size,accuracy,precision,recall,auc,f1_test,coef
time,,,,,,,,,
3.139770,LogisticRegression,"LogisticRegression(C=0.31622776601683794, rand...","[54064, 27022, 27015]",0.804331,0.805835,0.804331,0.867050,0.804419,"[0.18086648430125002, -0.74244441760028, 0.481..."
184.635613,RandomForestClassifier,"RandomForestClassifier(max_depth=10, min_sampl...","[54064, 27022, 27015]",0.806293,0.807668,0.806293,0.877194,0.806383,[None]
8134.301625,SVC,"SVC(C=0.01, gamma=0.1, probability=True, rando...","[54064, 27022, 27015]",0.801888,0.802470,0.801888,0.868395,0.801977,[None]
210.018007,GradientBoostingClassifier,"GradientBoostingClassifier(max_depth=9, max_fe...","[54064, 27022, 27015]",0.807292,0.808263,0.807292,0.877885,0.807387,[None]
324.349429,2 layer NN,"{'clf__batch_size': 100, 'clf__epochs': 25}","[54064, 27022, 27015]",0.805886,0.807257,0.805886,0.877252,0.805976,[None]


In [13]:
res.to_clipboard()

## SHAPLEY VALUE ##

In [33]:
import shap

ModuleNotFoundError: No module named 'shap'